In [9]:
import pandas as pd
import math
from datetime import datetime, timedelta
import torch
import joblib
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Definir la arquitectura del modelo
class DotacionPersonalNN(torch.nn.Module):
    def __init__(self):
        super(DotacionPersonalNN, self).__init__()
        self.layer1 = torch.nn.Linear(4, 64)
        self.layer2 = torch.nn.Linear(64, 32)
        self.layer3 = torch.nn.Linear(32, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Cargar el modelo y el scaler
model_path = 'model.pth'
scaler_path = 'scaler.pkl'

model = DotacionPersonalNN()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

scaler = joblib.load(scaler_path)

def generar_predicciones_masivas(suc_ids, fecha_inicio, fecha_fin, model, scaler, db_connection_string):
    try:
        # Crear conexión a la base de datos
        engine = create_engine(db_connection_string)

        # Generar rango de fechas
        fechas_a_validar = pd.date_range(fecha_inicio, fecha_fin).strftime('%Y-%m-%d').tolist()

        # Cargar transacciones por sucursal y fecha
        transacciones_query = """
        SELECT suc_id, fecha_id, hora_id, MAX(conteo_transacciones) AS max_transacciones
        FROM BI_Kielsa_Hecho_ProyeccionVenta_SucCanHora
        WHERE suc_id IN ({}) AND fecha_id BETWEEN '{}' AND '{}'
        GROUP BY suc_id, fecha_id, hora_id
        """.format(",".join(map(str, suc_ids)), fecha_inicio, fecha_fin)
        transacciones = pd.read_sql(transacciones_query, engine)
    
        # Estructura para almacenar los resultados
        resultados = []

        for suc_id in suc_ids:
            for fecha_validada in fechas_a_validar:
                dia_semana = datetime.strptime(fecha_validada, "%Y-%m-%d").weekday() + 1

                transacciones_dia = transacciones[(transacciones['suc_id'] == suc_id) & 
                                                (transacciones['fecha_id'] == fecha_validada)]
                
                print(f"Fecha: {fecha_validada}, Suc_Id: {suc_id}, Transacciones: {transacciones_dia.shape[0]}")
                
                if transacciones_dia.empty:
                    print(f"No hay transacciones para la sucursal {suc_id} en la fecha {fecha_validada}")
                
                for _, transaccion in transacciones_dia.iterrows():
                    print("hello")
                    hora = transaccion['hora_id']
                    transacciones_totales = math.ceil(transaccion['max_transacciones'])

                    # Preparar datos para el modelo
                    input_data = pd.DataFrame({
                        'Suc_Id': [suc_id],
                        'Dia_Semana': [dia_semana],
                        'Hora': [hora],
                        'Transacciones_Totales': [transacciones_totales]
                    })
                    
                    print(input_data)
                    input_scaled = scaler.transform(input_data.values)
                    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).view(1, -1)

                    # Predecir personal necesario
                    with torch.no_grad():
                        personal_necesario = model(input_tensor).item()
                        personal_necesario = math.ceil(personal_necesario) if personal_necesario >= 1.2 else math.floor(personal_necesario)

                    # Agregar resultados al dataframe
                    resultados.append({
                        "Fecha_Id": fecha_validada,
                        "Hora": hora,
                        "Dia_Semana": dia_semana,
                        "Cantidad_Transacciones": transacciones_totales,
                        "Personal_Necesario": max(1, personal_necesario),
                        "Suc_Id": suc_id
                    })

        # Convertir resultados a DataFrame
        resultados_df = pd.DataFrame(resultados)
        return resultados_df

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Parámetros de ejemplo
suc_ids = [1, 2, 3]  # IDs de las sucursales
fecha_inicio = "2025-02-01"
fecha_fin = "2025-02-28"
connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)
# Llamar a la función
resultados_df = generar_predicciones_masivas(suc_ids, fecha_inicio, fecha_fin, model, scaler, connection_string)

# save to csv
resultados_df.to_csv('resultados.csv', index=False)

/var/folders/5s/y64xy8cd6cd355kr26pngsrh0000gn/T/ipykernel_65027/1777695927.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path,

{'dia': 6, 'fecha': '2025-02-01', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 7, 'fecha': '2025-02-02', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 1, 'fecha': '2025-02-03', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 2, 'fecha': '2025-02-04', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 3, 'fecha': '2025-02-05', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 4, 'fecha': '2025-02-06', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 5, 'fecha': '2025-02-07', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 6, 'fecha': '2025-02-08', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 7, 'fecha': '2025-02-09', 'transacciones': <generator object DataFrame.iterrows at 0x15ca72ab0>}
{'dia': 1, 'fecha': '2025-02-10', 'transacciones': <gen

In [ ]:
# Cargar datos al dwf

connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/IA_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)

# Crear conexión a la base de datos
engine = create_engine(connection_string)

# Insertar datos en la tabla
resultados_df.to_sql('IA_Farinter_PrediccionPersonal', engine, if_exists='append', index=False)
